In [ ]:
%%writefile app.py
import streamlit as st
import joblib
import numpy as np
import shap
import matplotlib.pyplot as plt

# load model and vectorizer
model = joblib.load("xgb_fake_job_model.pkl")
vectorizer = joblib.load("tfidf_vectorizer.pkl")

# Initialize SHAP
shap.initjs()
explainer = shap.Explainer(model)

# streamlit UI
st.title("Fake Job Post Detector")
st.write("Paste a job posting text below to find out if it is potentially fake.")

user_input = st.text_area("Enter Job Posting Text:", height=300)

if st.button("Predict"):
    if user_input.strip() == "":
        st.warning("Please enter some text to analyze.")
    else:
        # vectorize input
        input_vector = vectorizer.transform([user_input])

        # make prediction
        prediction = model.predict(input_vector)[0]
        proba = model.predict_proba(input_vector)[0]

        # show prediction result
        if prediction == 1:
            st.error(f"This job post is likely **FAKE** (Confidence: {proba[1]*100:.2f}%)")
        else:
            st.success(f"This job post is likely **REAL** (Confidence: {proba[0]*100:.2f}%)")

        # SHAP explanation
        st.write("### 🔍 SHAP Explanation")
        try:
            # SHAP expects dense input for some models
            input_dense = input_vector.toarray()
            shap_values = explainer(input_dense)

            # add actual feature names to SHAP values
            feature_names = vectorizer.get_feature_names_out().tolist()
            shap_values.feature_names = feature_names

            # waterfall plot for a single sample
            fig, ax = plt.subplots(figsize=(10, 5))
            shap.plots.waterfall(shap_values[0], show=False, max_display=15)
            st.pyplot(fig)

        except Exception as e:
            st.warning("SHAP explanation failed. Reason: " + str(e))

In [ ]:
!ngrok config add-authtoken YOUR_NGROK_AUTHTOKEN 

Run Streamlit via ngrok:

In [21]:
from pyngrok import ngrok
import time
import threading

# function to run Streamlit app
def run_streamlit():
    !streamlit run app.py

# start streamlit in a separate thread
thread = threading.Thread(target=run_streamlit)
thread.start()

# wait a sec for streamlit to start
time.sleep(5)

# Open a tunnel on port 8501
public_url = ngrok.connect(8501)
print("Your app is live at:", public_url)